# 0. Dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import plotly.graph_objects as go 
import plotly.express as px #for colors

#prices
from datetime import datetime
import logging
from entsoe import EntsoePandasClient
import pandas as pd
import itertools

# 1. Data load

## 1.1 Fluvius

### Consumption

In [ ]:
#explain how Fluvius data has been used, correlation, clustering?,graphs, etc
consumption = pd.read_csv('data/consumption.csv')
#in kWh by default see legend doc

In [ ]:
#remove stat_time consumption
consumption = consumption.drop(columns=['start_time'])

In [ ]:
consumption #do only for one profile, temporary

Here could add boxplot

In [ ]:
#for id == 173 make a numpy array with consumption column
cons_1 = consumption[consumption['id'] == 272] #low
cons_1 = cons_1.groupby(['date']).sum()
# cons_1 = cons_1['consumption'].to_numpy()
cons_2 = consumption[consumption['id'] == 173] #median
cons_2 = cons_2.groupby(['date']).sum()
# cons_2 = cons_2['consumption'].to_numpy()
cons_3 = consumption[consumption['id'] == 550] #high
cons_3 = cons_3.groupby(['date']).sum()
# cons_3 = cons_3['consumption'].to_numpy()

In [ ]:
light_green = '#228A83'
dark_green = '#2E5651'
blue = '#002060'

In [ ]:
#plot the df_grouped with colors light_green and blue (earlier defined) using plotly
fig = go.Figure()
# fig.add_trace(go.Scatter(x=cons_1.index, y=cons_1['consumption'], mode='lines', name='Low', line=dict(color=dark_green)))
fig.add_trace(go.Scatter(x=cons_1.index, y=cons_2['consumption'], mode='lines', name='median', line=dict(color=blue)))
# fig.add_trace(go.Scatter(x=cons_1.index, y=cons_3['consumption'], mode='lines', name='high', line=dict(color=light_green)))
fig.update_layout(
    title={
        'text': "Consumption profiles Fluvius",
        'font': {
            'color': '#2E5651'
        }
    },
    xaxis_title='Date',
    yaxis_title='kWh per day'
)
fig.show()

In [ ]:
#in cons_1 print the sum of consumption and the stad dev
print('Low consumption profile')
print('Sum of consumption:', cons_1['consumption'].sum()) 
print('Standard deviation:', cons_1['consumption'].std())
print('Medium consumption profile')
print('Sum of consumption:', cons_2['consumption'].sum())
print('Standard deviation:', cons_2['consumption'].std())
print('High consumption profile')
print('Sum of consumption:', cons_3['consumption'].sum())
print('Standard deviation:', cons_3['consumption'].std())

In [ ]:
len(cons_3) #quarterly data, all same length

In [ ]:

#cons_1
print('Average:', cons_1['consumption'].mean())
print('Max:', cons_1['consumption'].max())
print('Min:', cons_1['consumption'].min())
print('Standard deviation:', cons_1['consumption'].std())
print('Count:', cons_1['consumption'].count())
#cons_2
print('Average:', cons_2['consumption'].mean())
print('Max:', cons_2['consumption'].max())
print('Min:', cons_2['consumption'].min())
print('Standard deviation:', cons_2['consumption'].std())
print('Count:', cons_2['consumption'].count())
#cons_3
print('Average:', cons_3['consumption'].mean())
print('Max:', cons_3['consumption'].max())
print('Min:', cons_3['consumption'].min())
print('Standard deviation:', cons_3['consumption'].std())
print('Count:', cons_3['consumption'].count())

### Production

In [ ]:
efficiency_loss = 0.3 #tocheck... impact as ell on profile, probably a bit lower
#explain where weather data has been generated

In [ ]:
#load weather data
weather = pd.read_csv('data/weather.csv')

#make columns now for each profile, create first column and then for loop 1 to 4...

In [ ]:
weather

In [ ]:
#remove start_time
weather = weather.drop(columns=['start_time'])

In [ ]:
#groupe weather by date and take the mean
weather_sum = weather.groupby(['date']).sum()
weather_mean = weather.groupby(['date']).mean()

In [ ]:
fig = go.Figure()
# Adding Solar Irradiation Trace
fig.add_trace(go.Scatter(x=weather_sum.index, y=weather_sum['solar_irradiation'], mode='lines', name='Solar irradiation', line=dict(color=light_green)))

# Adding Temperature Trace, setting it to the secondary y-axis
fig.add_trace(go.Scatter(x=weather_sum.index, y=weather_mean['temperature'], mode='lines', name='Temperature', line=dict(color='gray'), yaxis='y2'))

# Updating layout to include a secondary y-axis
fig.update_layout(
    xaxis=dict(title='Date'),
    yaxis=dict(title='Solar Irradiation (W/m²)'),
    yaxis2=dict(title='Temperature (°C)', overlaying='y', side='right'),
    title={
        'text': "Weather data needed for PV production predictions",
        'font': {
            'color': '#2E5651'
        }
    },
)

fig.show()

#plot 

In [ ]:
def calc_prod(df, efficiency_loss): #no need area panel
    #use temp and irradiance to calculate the production in df
    df['prod_one_kw_panel'] = (1 - 0.005 * (df['temperature'] - 25)) * df['solar_irradiation'] * efficiency_loss #make it profile name!!
    print('Done')
    return df

weather_with_prod = calc_prod(weather, efficiency_loss)

In [ ]:
#print sum of prod_one_kw_panel
print('Total production of 1 kW panel:', weather_with_prod['prod_one_kw_panel'].sum()/1000, 'kWh')
print('Total irrandiance in BE:', weather_with_prod['solar_irradiation'].sum()/1000, 'kWh/m²/year') #W/m² is here one hour timestep, is an instant (average here) delivery of power, not energy
#The yearly solar irradiance in Belgium typically varies around 1,000 to 1,150 kWh per square meter (kWh/m²/year)

#Average consumption in Belgium is 4-5 persons 7,500 kWh/year (profile 2?)

In [ ]:
area_panel_1 = 1
area_panel_2 = 5
area_panel_3 = 15

In [ ]:
#create a column prod_1, prod_2, prod_3 in weather_with_prod by multiplying prod_one_kw_panel with area_panel_1, area_panel_2, area_panel_3
weather_with_prod['prod_1'] = weather_with_prod['prod_one_kw_panel'] * area_panel_1
weather_with_prod['prod_2'] = weather_with_prod['prod_one_kw_panel'] * area_panel_2
weather_with_prod['prod_3'] = weather_with_prod['prod_one_kw_panel'] * area_panel_3

In [ ]:
weather_with_prod

In [ ]:
#print sum prod_1
print('Total production of 1 kW panel:', weather_with_prod['prod_1'].sum()/1000, 'kWh')
print('Total production of 5 kW panel:', weather_with_prod['prod_2'].sum()/1000, 'kWh')
print('Total production of 15 kW panel:', weather_with_prod['prod_3'].sum()/1000, 'kWh')

In [ ]:
#delete start_time from weather_with_prod
# weather_with_prod = weather_with_prod.drop(columns=['start_time', 'solar_irradiation', 'temperature'])
weather_with_prod = weather_with_prod.drop(columns=['solar_irradiation', 'temperature'])

#group weather_with_prod by date and take the sum
weather_with_prod = weather_with_prod.groupby(['date']).sum()

In [ ]:
weather_with_prod

In [ ]:
#plot prod_1, prod_2, prod_3
fig = go.Figure()
fig.add_trace(go.Scatter(x=weather_with_prod.index, y=weather_with_prod['prod_1'], mode='lines', name='1 kW panel', line=dict(color=dark_green)))
fig.add_trace(go.Scatter(x=weather_with_prod.index, y=weather_with_prod['prod_2'], mode='lines', name='5 kW panel', line=dict(color=blue)))
fig.add_trace(go.Scatter(x=weather_with_prod.index, y=weather_with_prod['prod_3'], mode='lines', name='15 kW panel', line=dict(color=light_green)))
fig.update_layout(
    title={
        'text': "Consumption profiles Fluvius",
        'font': {
            'color': '#2E5651'
        }
    },
    xaxis_title='Date',
    yaxis_title='kWh'
)

In [ ]:
def make_array_from_df(df, col_name):
    return df[col_name].to_numpy()

In [ ]:
#take array from it and do 4 replication same length normally, divide uniformly and by 1000 (to get KWh per quarter)
#same as make_Wp_to_kWh function
prod_1 = make_array_from_df(weather_with_prod, 'prod_1')/1000
prod_2 = make_array_from_df(weather_with_prod, 'prod_2')/1000
prod_3 = make_array_from_df(weather_with_prod, 'prod_3')/1000

In [ ]:
#print total consumption per cons
print(f'sum cons 1: {cons_1.sum()}, sum cons 2: {cons_2.sum()}, sum cons 3: {cons_3.sum()} kWh/year')
print(f'sum prod 1: {prod_1.sum()}, sum prod 2: {prod_2.sum()}, sum prod 3: {prod_3.sum()} kWh/year')
#changed panel_1 and efficiency loss!!! maybe need to be increased

### Prices

In [ ]:
#read from file
prices_fluvius = np.loadtxt('data/prices_fluvius.csv', delimiter=',')
prices_fluvius #length is 8760

In [ ]:
import numpy as np


def rolling_average_by_chunks(data, chunk_size):
  """Averages data by chunk_size, returning an array of the same length.

  Args:
    data: A NumPy array of any dimension.
    chunk_size: The size of the rolling window for averaging.

  Returns:
    A NumPy array with the same dimensions as data, containing the rolling
    averages.
  """
  if chunk_size <= 1:
    raise ValueError(f"chunk_size must be greater than 1, got {chunk_size}")

  if len(data) % chunk_size != 0:
    raise ValueError(
        f"data length {len(data)} must be a multiple of chunk_size {chunk_size}"
    )

  # We can use np.lib.stride_tricks.sliding_window_view to create a 2D view of the
  # data that allows us to calculate the averages across chunks. However, this
  # method has a known issue where it doesn't support complex data types. As a
  # workaround, we can reshape the data to 2D if it has more than one dimension
  # and then reshape it back to its original shape after the calculation.
  if data.ndim > 1:
    data_reshaped = data.reshape(-1, data.shape[-1])
  else:
    data_reshaped = data

  # Create a 2D view of the data with overlapping windows of size chunk_size.
  data_strided = np.lib.stride_tricks.sliding_window_view(data_reshaped, window_shape=(chunk_size,))

  # Calculate the mean along the last dimension (axis=0) to get the rolling averages.
  averages = np.mean(data_strided, axis=0)

  if data.ndim > 1:
    averages = averages.reshape(data.shape)

  return averages

# Example usage:
# data = np.arange(72)
# chunk_size = 24
# output = rolling_average_by_chunks(data, chunk_size)

# print(data)

# print(output)


In [ ]:
prices_fluvius

In [ ]:
feb_1= [122.62,
    122.62, 119.30, 109.02, 88.72, 83.85, 109.53, 125.38, 173.68,
    188.11, 194.25, 178.99, 165.76, 145.10, 125.00, 134.32, 140.00,
    141.91, 167.62, 197.74, 183.60, 202.10, 140.35, 163.23, 127.11, 127.11
]
may_1 = [109.9,
    109.90, 99.90, 95.49, 90.88, 87.61, 88.00, 87.44, 91.89,
    90.49, 88.50, 81.29, 67.50, 54.45, 34.69, 30.00, 41.12,
    51.80, 49.33, 89.42, 109.55, 123.35, 121.72, 117.90, 111.04, 111.04 
]

aug_1 = [68.47,
    68.47, 58.89, 57.86, 56.94, 57.90, 63.40, 75.34, 90.91,
    90.58, 88.18, 77.00, 67.07, 67.10, 61.96, 53.09, 48.26,
    54.60, 67.94, 79.59, 95.05, 106.41, 112.42, 102.55, 92.78, 92.78
]

nov_1 = [65.31,
    65.31, 64.20, 50.23, 26.48, 23.75, 30.13, 47.99, 68.07,
    66.88, 69.23, 46.22, 43.50, 47.64, 38.92, 32.84, 37.28,
    51.58, 63.40, 80.46, 71.80, 57.48, 40.15, 40.77, 23.00, 23.00
]
#create list from 0 to 23
time_intervals = list(range(24))


In [ ]:
import plotly.graph_objects as go

light_green = '#228A83'
dark_green = '#2E5651'
blue = '#002060'

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_intervals, y=feb_1, mode='lines', name='February', line=dict(color=light_green, shape='vh')))
fig.add_trace(go.Scatter(x=time_intervals, y=may_1, mode='lines', name='May', line=dict(color=blue, shape='vh')))
fig.add_trace(go.Scatter(x=time_intervals, y=aug_1, mode='lines', name='August', line=dict(color=dark_green, shape='vh')))
fig.add_trace(go.Scatter(x=time_intervals, y=nov_1, mode='lines', name='November', line=dict(color='gray', shape='vh')))

fig.update_layout(
    title={
        'text': "Day-ahead_prices Belgium in 2023",
        'font': {
            'color': '#2E5651'
        }
    },
    xaxis_title='Time intervals (hours)',
    yaxis_title='Price (€/MWh)'
)
fig.show()


In [ ]:
#plot feb_1, may_1, aug_1, nov_1 with time intervals
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_intervals, y=feb_1, mode='lines', name='February', line=dict(color=dark_green)))
fig.add_trace(go.Scatter(x=time_intervals, y=may_1, mode='lines', name='May', line=dict(color=blue)))
fig.add_trace(go.Scatter(x=time_intervals, y=aug_1, mode='lines', name='August', line=dict(color=light_green)))
fig.add_trace(go.Scatter(x=time_intervals, y=nov_1, mode='lines', name='November', line=dict(color='gray')))
fig.update_layout(
    title={
        'text': "Prices Fluvius",
        'font': {
            'color': '#2E5651'
        }
    },
    xaxis_title='Time intervals (hours)',
    yaxis_title='€/MWh'
)
fig.show()

## 1.2 robbe

In [ ]:
df = pd.read_csv('data/robbe.csv')

In [ ]:
df #output from other file

Er ontbreekt data maar blc - moet wel zorgen zelfde lengte voor analyysis later...

Also mistmatch on s'en branle

In [ ]:
#divide consumption and production data by 1000 to Kwh
df['consumption'] = df['consumption']/1000
df['production'] = df['production']/1000
#multiply consumption and production data by 1/12
df['consumption'] = df['consumption']*1/12
df['production'] = df['production']*1/12


In [ ]:
#print sum of consumption and production
print('sum of consumption:', df['consumption'].sum())
print('sum of production:', df['production'].sum())

In [ ]:
#make time column datetime
df['time'] = pd.to_datetime(df['time'])

In [ ]:
#make in df a new hour column, day of the week and month
df['hour'] = pd.to_datetime(df['time']).dt.hour
df['day'] = pd.to_datetime(df['time']).dt.day_name()
df['month'] = pd.to_datetime(df['time']).dt.month_name()

In [ ]:
df

In [ ]:
df['date'] = df['time'].dt.date.astype(str)

In [ ]:
df

Make first adjustments for dates missing, done in csv... so load these on Github (no excel)

31 March, Mai, minuit rajouté et que 10 Février jusqu'au 30 Juin

Visuals from here

In [ ]:
# #for each column consumption and column production print average, max and min
# print('Consumption')
# print('Average:', df['consumption'].mean())
# print('Max:', df['consumption'].max())
# print('Min:', df['consumption'].min())
# print('Standard deviation:', df['consumption'].std())
# print('Count:', df['consumption'].count())
# print('Production')
# print('Average:', df['production'].mean())
# print('Max:', df['production'].max())
# print('Min:', df['production'].min())
# print('Standard deviation:', df['production'].std())
# print('Count:', df['production'].count())

count is not exact but blc

In [ ]:
#print the number of unique values in df['date']
print('Number of unique values in date:', df['date'].nunique())

In [ ]:
#create df_merged from df removing hour, time
df_merged = df.drop(['hour', 'time', 'day', 'month'], axis=1)

In [ ]:
#create df_grouped which is the averaged grouped by date
df_grouped = df_merged.groupby(['date']).sum()  

In [ ]:
df_grouped

# Correlation (one cell)

In [ ]:
#take only date 

The use of kW (kilowatts) rather than kWh (kilowatt-hours) is suggested because the data points seem to represent the instantaneous power production or consumption at each timestamp, not the cumulative energy used or produced over time, which would be measured in kWh.

In [ ]:
#plot the df_grouped with colors light_green and blue (earlier defined) using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_grouped.index, y=df_grouped['consumption'], mode='lines', name='Consumption', line=dict(color=light_green)))
fig.add_trace(go.Scatter(x=df_grouped.index, y=df_grouped['production'], mode='lines', name='Production', line=dict(color=blue)))
fig.update_layout(
    title={
        'text': "Daily consumption and production Robbe",
        'font': {
            'color': '#2E5651'
        }
    },
    xaxis_title='Date',
    yaxis_title='kWh per day'
)
fig.show()

In [ ]:
#for each column consumption and column production print average, max and min
#sum
print('sum of consumption:', df_grouped['consumption'].sum())
print('sum of production:', df_grouped['production'].sum())
print('Consumption')
print('Average:', df_grouped['consumption'].mean())
print('Max:', df_grouped['consumption'].max())
print('Min:', df_grouped['consumption'].min())
print('Standard deviation:', df_grouped['consumption'].std())
#print volatility
print('Volatility:', df_grouped['consumption'].std()/df_grouped['consumption'].mean()*100)
print('Count:', df_grouped['consumption'].count())
print('Production')
print('Average:', df_grouped['production'].mean())
print('Max:', df_grouped['production'].max())
print('Min:', df_grouped['production'].min())
print('Standard deviation:', df_grouped['production'].std())
#print volatility
print('Volatility:', df_grouped['production'].std()/df_grouped['production'].mean()*100)
print('Count:', df_grouped['production'].count())

Then visualisation of production and consumption

# Prices

No need to run and blc last cell already preprocess, no need visualize prices

In [ ]:
class DayAheadPrices:
    def __init__(self, api_key):
        self.client = EntsoePandasClient(api_key=api_key)

    def fetch_and_print_day_ahead_prices(self, start_date, end_date, country_code='BE'):
        """Fetches and prints the day ahead prices for a specified country within a date range."""
        # Convert start_date and end_date to pandas Timestamp with timezone
        timezone = 'Europe/Brussels'
        date_start = pd.Timestamp(start_date, tz=timezone)
        date_end = pd.Timestamp(end_date, tz=timezone)

        # logging.info(f'Fetching day ahead prices from ENTSO-E for {country_code}: {date_start} to {date_end}')
        
        # Fetch day ahead prices
        try:
            day_ahead_prices_df = self.client.query_day_ahead_prices(country_code=country_code, start=date_start, end=date_end)
            # Convert the DataFrame to a dictionary with the date as the key and a list of prices as the value
            prices_by_day = {}
            for date, group in day_ahead_prices_df.groupby(day_ahead_prices_df.index.date):
                prices_by_day[str(date)] = group.values.tolist()
                
            print("Day Ahead Prices (€/MWh):")
            print(prices_by_day)
            return prices_by_day
        except Exception as e:
            print(f"An error occurred while fetching data: {e}")


api_key = 'eaccd9d9-2a8e-42b0-9257-2aa47096934e' # Replace with your actual API key
start_date = '2023-02-10'
end_date = '2023-06-30'
#create DayAheadPrices elem
elem = DayAheadPrices(api_key)
robbe_prices = DayAheadPrices.fetch_and_print_day_ahead_prices(elem,start_date, end_date)

In [ ]:
#remove 31-03-2023 and 31-05-2023 from robbe_prices
robbe_prices.pop('2023-03-31', None)
robbe_prices.pop('2023-05-31', None)

In [ ]:
robbe_prices

In [ ]:
len(robbe_prices)

In [ ]:
def create_long_array(prices_dict):
    # Concatenate the lists of prices into a single numpy array
    all_prices = np.concatenate([np.array(prices_dict[date]) for date in prices_dict])
    return all_prices

# Applying the function to 'robbe_prices' to create the numpy array
robbe_prices_array = create_long_array(robbe_prices)

In [ ]:
len(robbe_prices_array)

In [ ]:
robbe_prices

In [ ]:
def filter_prices_within_date_range(prices, start_date, end_date):
    # Convert string dates to datetime objects for comparison
    start_date_dt = datetime.strptime(start_date, '%Y/%m/%d')
    end_date_dt = datetime.strptime(end_date, '%Y/%m/%d')

    # Create a new dictionary to store the filtered prices within the date range
    filtered_prices_within_date_range = {
        date: price for date, price in prices.items()
        if start_date_dt <= datetime.strptime(date, '%Y-%m-%d') <= end_date_dt
    }

    return filtered_prices_within_date_range

# filtered_robbe_prices = filter_prices_within_date_range(robbe_prices, '2023/02/09', '2023/06/30')


# prices_values = list(prices.values())
# prices_values = prices_values[:-1]
# prices_values_one_list = [sublist[:24] + [0] * (24 - len(sublist)) if len(sublist) > 24 else sublist + [0] * (24 - len(sublist)) for sublist in prices_values]
# prices_values_one_list = [item for sublist in prices_values_one_list for item in sublist]

# #make numpy
# prices_values_one_list = np.array(prices_values_one_list)
# #dvide by 1000 to get Eur/kWh
# prices_values_one_list = prices_values_one_list/1000

# def length_distribution(list_of_lists):
#     length_count = {}
#     for i, sublist in enumerate(list_of_lists, start=1):
#         length = len(sublist)
#         if length in length_count:
#             length_count[length].append(i)
#         else:
#             length_count[length] = [i]
    
#     # Print the length distribution
#     print("Length Distribution:")
#     for length, indices in sorted(length_count.items()):
#         print(f"Length {length}: {len(indices)} sublists at indices {indices}")

# length_distribution(prices_values) #remove last elem and put zero or remove last.
